In [4]:
import math
from os import path, walk
from json import dumps, loads
from pyvis.network import Network
from dataclasses import dataclass, field

from attack import Attack
from defend import DTechnique, DSystem, Defend

In [5]:
d = Defend(Attack(local=True))

In [28]:
Out = './Poligon'
net = Network(notebook=True, height='1000px', width='1200px', select_menu=False, filter_menu=True, heading='A')

In [29]:
net.add_node('A', color='#FFF')

In [30]:
net.show('Test.html')

Test.html
